In [1]:
from validate_adrresses import validate_address
import pandas as pd
import os
from tqdm import tqdm
print(os.getcwd())

/home/luis/Documents/Belcorp/iteration_adresses


In [2]:
df = pd.read_csv('./data/direcciones.csv', sep=';')

/tmp/ipykernel_3261625/4095483860.py:1: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/direcciones.csv', sep=';')


In [3]:
cols = [col for col in df]
col1 = cols[:3]
col2 = cols[3:]

cols = []
for col in col1:
    cols.append(col)
cols.append("NUEVA DIRECCION")
for col in col2:
    cols.append(col)

In [4]:
df["NUEVA DIRECCION"] = None
df = df.reindex(columns=cols)
df = df[df["IND TERRITORIOS"] == 9]

In [5]:
df_tmp = df[["DIRECCION DE DOMICILIO", "COMUNA DOMICILIO", "NUEVA DIRECCION", "X", "Y"]]
df_tmp = df_tmp[3000:4000]
df_tmp 

,DIRECCION DE DOMICILIO,COMUNA DOMICILIO,NUEVA DIRECCION,X,Y
15021,"ARAUCO 51 , LA CRUZ LA CRUZ",HIJUELAS,None,,
15023,ESCOCIA 1300 POB A ALARCON DEL CANTO,HUALPEN,None,,
15024,CAMPANARIO 612 CASA PORTON ROJO,LAS CONDES,None,,
15034,ESMERALDA #2797 ANTOFAGASTA,ANTOFAGASTA,None,,
15040,LA VENDIMIA 2092 P. MEDIA LUNA PEÑALOLEN P. ME...,PEÑALOLEN,None,,
...,...,...,...,...,...
19694,"LAS GOLONDRINAS CASA ESQUINA, FRETE SEDE COMUN...",SANTA MARIA,None,,
19698,ATLANTA 830 VILLA LAS AMERICAS V,LOS ANGELES,None,,
19699,CAUPOLICAN CAMPAMENTO AL FINAL DE CALLE CAUPOL...,COLLIPULLI,None,-32.7790545,-70.9654515
19701,C. UNIHUE AL LADO DE NEGOCIO GABY,MAULE,None,,


In [6]:
import dask.dataframe as dd
import pandas as pd
import numpy as np

ddata = dd.from_pandas(df_tmp, npartitions=30)

# Sample meta dataframe to indicate the structure of the output
meta = pd.DataFrame({
    "DIRECCION DE DOMICILIO": pd.Series(dtype='str'),
    "COMUNA DOMICILIO": pd.Series(dtype='str'),
    "NUEVA DIRECCION": pd.Series(dtype='str'),
    "X": pd.Series(dtype='float'),
    "Y": pd.Series(dtype='float')
})

def apply_validate_address(df_chunk):
    # Handle missing values before passing to validate_address
    df_chunk[["DIRECCION DE DOMICILIO", "COMUNA DOMICILIO"]] = df_chunk[["DIRECCION DE DOMICILIO", "COMUNA DOMICILIO"]].fillna("")
    
    # Apply the validation function to each row and unpack the results
    df_chunk[["NUEVA DIRECCION", "X", "Y"]] = df_chunk.apply(
        lambda row: pd.Series(validate_address(row["DIRECCION DE DOMICILIO"] + " " + row["COMUNA DOMICILIO"])),
        axis=1
    )
    return df_chunk

# Map the function to each partition, with the meta keyword to define the structure
res = ddata.map_partitions(apply_validate_address, meta=meta).compute()

['DIAMANTE', '4990', 'PUERTO', 'MONTT', 'LOS LAGOS', 'VARAS']  ------  DIAMANTE 4990.PUERTO MONTT.LOS LAGOS. PUERTO VARAS  -------  4990, Diamante, La Casa de Mi Sueño, Puerta Sur, Puerto Montt, Provincia de Llanquihue, Región de Los Lagos, 5480000, Chile
['EL BOSQUE', '10674', 'LOS MANANTIALES', 'CON', 'PASAJE', 'LA PINTANA']  ------  EL BOSQUE 10674 LOS MANANTIALES CON PASAJE EL BOSQUE LA PINTANA  -------  Pasaje Los Manantiales, Vertientes de Curicó, Barros Negros, Curicó, Provincia de Curicó, Región del Maule, 3340814, Chile
['CLAUDIO', 'ARRAU', 'LEON', 'CASA 684', 'VIÑA', 'DEL', 'MAR']  ------  CLAUDIO ARRAU LEON CASA 684 VIÑA DEL MAR  -------  Claudio Arrau León, Villa Oriente, Reñaca Alto, Viña del Mar, Provincia de Valparaíso, Región de Valparaíso, 2520000, Chile
sin coincidencias ['CINCO', '#9203', 'PUDAHUEL']  ------  CINCO #9203 PUDAHUEL PUDAHUEL
sin coincidencias ['SAN ANTONIO', 'NALTAGUA', 'ISLA', 'MAIPO']  ------  SAN ANTONIO DE NALTAGUA SN ISLA DE MAIPO
sin coincidencias

In [7]:
res

,DIRECCION DE DOMICILIO,COMUNA DOMICILIO,NUEVA DIRECCION,X,Y
15021,"ARAUCO 51 , LA CRUZ LA CRUZ",HIJUELAS,None,NaN,NaN
15023,ESCOCIA 1300 POB A ALARCON DEL CANTO,HUALPEN,"Embassy of Chile, ซอยสุขุมวิท 16, สุขุมวิท, แข...",13.731063,100.560631
15024,CAMPANARIO 612 CASA PORTON ROJO,LAS CONDES,None,NaN,NaN
15034,ESMERALDA #2797 ANTOFAGASTA,ANTOFAGASTA,None,NaN,NaN
15040,LA VENDIMIA 2092 P. MEDIA LUNA PEÑALOLEN P. ME...,PEÑALOLEN,"La Vendimia Sur, Barrio El Solar, Peñalolén, P...",-33.485876,-70.555197
...,...,...,...,...,...
19694,"LAS GOLONDRINAS CASA ESQUINA, FRETE SEDE COMUN...",SANTA MARIA,"Las Golondrinas, Santa Filomena, Jahuel, Santa...",-32.696620,-70.609211
19698,ATLANTA 830 VILLA LAS AMERICAS V,LOS ANGELES,"Villa Las Américas, Limache, Provincia de Marg...",-32.980383,-71.276365
19699,CAUPOLICAN CAMPAMENTO AL FINAL DE CALLE CAUPOL...,COLLIPULLI,"Caupolican, Campamento Millantu, Puente Alto, ...",-33.612448,-70.513372
19701,C. UNIHUE AL LADO DE NEGOCIO GABY,MAULE,None,NaN,NaN


In [8]:
print("A")

A


In [9]:
res.to_csv('./data/out/ad4.csv')